In [1]:
# Import data into pandas dataframe
import pandas as pd
df_train = pd.read_excel('relevance_train_processed.xlsx')
df_test = pd.read_excel('relevance_test_processed.xlsx')

In [2]:
#Import other relevant libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#Make all nan values into strings
df_train_str = df_train.where(pd.notnull(df_train),'')
df_test_str = df_test.where(pd.notnull(df_test),'')

In [4]:
#For this simple benchmark model only accounting for body
#Separate training data into content/features (x), and judgement/label (y)
x_train = df_train_str['body']
y_train = df_train_str['judgement']
x_test = df_test_str['body'] 

#For training purposes separate training into learning and trialing data, trial size 20%
#Important even if no training for consistency purposes when measuring accuracy
x_learn, x_trial, y_learn, y_trial_real = train_test_split(x_train, y_train, test_size = 0.2, random_state=42)

In [5]:
#Evaluation against all 0 bucket
buckets = [0] * len(list(y_trial_real)) # generate list of all 0 predictions
print(accuracy_score(y_trial_real, buckets))

0.8345170454545454


In [7]:
#Output CSV with results
prediction_pd = pd.DataFrame({'id': df_test_str['id'], 'judgement': [0] * len(list(df_test_str['id']))})
prediction_pd.to_csv('SparseBenchmark.csv', index = False)

In [ ]:
#For report purposes run model with multiple seeds
cycles = 10 #initiate as desired with int (there is a max iter for sklearn)
accuracy_arr = [0] * cycles
for i in range(cycles):
    x_learn, x_trial, y_learn, y_trial_real = train_test_split(x_train, y_train, test_size = 0.2, random_state=i)
    y_trial_predicted = standard_benchmark(x_learn, y_learn, x_trial)
    accuracy_arr[i] = accuracy_score(buckets, y_trial_real)

#Print accuracy average and standard deviation
print('Average Accuracy:', sum(accuracy_arr)/len(accuracy_arr))
print('Standard Deviation:', statistics.pstdev(accuracy_arr))